In [1]:
import argparse
import os
import warnings
import yaml

import pytorch_lightning as pl
import torch
import numpy as np
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.accelerators import find_usable_cuda_devices
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from nanodet.trainer.pseudo_lab_task import PseudoLabelTrainingTask
from nanodet.trainer.dist_task import DistTrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)

#Set logger and seed
logger = NanoDetLightningLogger('test')
pl.seed_everything(9)

#Function to create the task configuration file required for training
def create_exp_cfg(yml_path, task):
    all_names = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    #Load the YAML file
    with open(yml_path, 'r') as file:
        temp_cfg = yaml.safe_load(file)
    #Save dir of the model
    temp_cfg['save_dir'] = 'models/task' + str(task)
    #If base task, training and testing classes are the same
    if task == 0:
        temp_cfg['data']['train']['class_names'] = all_names[:15]
        temp_cfg['data']['val']['class_names'] = all_names[:15]
        temp_cfg['model']['arch']['head']['num_classes'] = 20 #15
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20 #15
    #Else, training only on task specific class, and testing on all classes
    else:
        temp_cfg['data']['train']['class_names'] = [all_names[14+task]]
        temp_cfg['data']['val']['class_names'] = all_names[:15+task]
        temp_cfg['model']['arch']['head']['num_classes'] = 20#15+task
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20#15+task
        temp_cfg['schedule']['load_model'] = 'models/task' + str(task-1) + '/model_last.ckpt'
        
    temp_cfg_name = 'cfg/task' + str(task) + '.yml'
    print(temp_cfg_name)
    #Save the new configuration file
    with open(temp_cfg_name, 'w') as file:
        yaml.safe_dump(temp_cfg, file)

Global seed set to 9


In [2]:
###LEARNING STREAM###
#task 0: train on first 15 classes, test on 15 classes
#task 1: train on class n°16, test on 16 classes
#task 2: train on class n°17, test on 17 classes
#task 3: train on class n°18, test on 18 classes
#task 4: train on class n°19, test on 19 classes
#task 5: train on class n°20, test on 20 classes
#torch.set_printoptions(profile="full")
#opt_epochs = [60, 80, 40, 60 ,40]
for task in range (1, 6):
    logger = NanoDetLightningLogger('run_logs/task'+str(task))
    logger.info("Starting task" + str(task))
    logger.info("Setting up data...")
    #Create the task configuration file based on the task number and load the configuration
    create_exp_cfg('cfg/VOC.yml', task)
    load_config(cfg, 'cfg/task' + str(task) + '.yml')
    #Build datasets and dataloaders based on the task configuration file
    train_dataset = build_dataset(cfg.data.train, "train")
    #If task is not 0, create the replay dataset using the buffer
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=True,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=True,
    )
    val_dataset = build_dataset(cfg.data.val, "test")
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=False,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=False,
    )
    evaluator = build_evaluator(cfg.evaluator, val_dataset)
    
    #Create the model based on the task configuration file
    logger.info("Creating models")
    if task == 0:
        TrainTask = TrainingTask(cfg, evaluator)
    else:
        TrainTask = PseudoLabelTrainingTask(cfg, evaluator)
        #Load the model weights if task is not 0
        if "load_model" in cfg.schedule:
            ckpt = torch.load(cfg.schedule.load_model)
            load_model_weight(TrainTask.model, ckpt, logger)
            logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))
    
    model_resume_path = (
        os.path.join(cfg.save_dir, "model_last.ckpt")
        if "resume" in cfg.schedule
        else None
    )
    #Set the device to GPU if available
    if cfg.device.gpu_ids == -1:
        logger.info("Using CPU training")
        accelerator, devices, strategy, precision = (
            "cpu",
            None,
            None,
            cfg.device.precision,
        )
    else:
        accelerator, devices, strategy, precision = (
            "gpu",
            cfg.device.gpu_ids,
            None,
            cfg.device.precision,
        )

    if devices and len(devices) > 1:
        strategy = "ddp"
        env_utils.set_multi_processing(distributed=True)

    trainer = pl.Trainer(
        default_root_dir=cfg.save_dir,
        max_epochs=100,
        check_val_every_n_epoch=10,
        accelerator=accelerator,
        devices=[2],
        log_every_n_steps=cfg.log.interval,
        num_sanity_val_steps=0,
        callbacks=[TQDMProgressBar(refresh_rate=0)],# TrainTask.early_stop_callback],
        logger=logger,
        benchmark=cfg.get("cudnn_benchmark", True),
        gradient_clip_val=cfg.get("grad_clip", 0.0),
        strategy=strategy,
        precision=precision,
    )
    trainer.fit(TrainTask, train_dataloader, val_dataloader, ckpt_path=model_resume_path)

[NanoDet][02-19 17:23:30]INFO:Starting task1
[NanoDet][02-19 17:23:30]INFO:Starting task1
[NanoDet][02-19 17:23:30]INFO:Setting up data...
[NanoDet][02-19 17:23:30]INFO:Setting up data...


cfg/task1.yml


Load 273 xml files and 625 boxes
creating index...
index created!


[NanoDet][02-19 17:23:30]INFO:Creating models
[NanoDet][02-19 17:23:30]INFO:Creating models
INFO:NanoDet:Creating models


Load 4530 xml files and 13606 boxes
creating index...
index created!
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


[NanoDet][02-19 17:23:32]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-19 17:23:32]INFO:Loaded model weight from models/task0/model_last.ckpt
INFO:NanoDet:Loaded model weight from models/task0/model_last.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name  | Type        | Params
--------------------------------------
0 | model | NanoDetPlus | 1.2 M 
--------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.699     Total estimated model params size (MB)
/home/pasti/anaconda3/envs/nanodet/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see lo

IndexError: too many indices for tensor of dimension 1

## DEBUGGING

In [4]:
task = 1
logger = NanoDetLightningLogger('run_logs/task'+str(task))
logger.info("Starting task" + str(task))
logger.info("Setting up data...")
#Create the task configuration file based on the task number and load the configuration
create_exp_cfg('cfg/VOC.yml', task)
load_config(cfg, 'cfg/task' + str(task) + '.yml')
#Build datasets and dataloaders based on the task configuration file
train_dataset = build_dataset(cfg.data.train, "train")
#If task is not 0, create the replay dataset using the buffer
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=True,
)
val_dataset = build_dataset(cfg.data.val, "test")
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=False,
)
evaluator = build_evaluator(cfg.evaluator, val_dataset)

#Create the model based on the task configuration file
logger.info("Creating models")
if task == 0:
    TrainTask = TrainingTask(cfg, evaluator)
else:
    TrainTask = DistTrainingTask(cfg, evaluator)
    #Load the model weights if task is not 0
    if "load_model" in cfg.schedule:
        ckpt = torch.load(cfg.schedule.load_model)
        load_model_weight(TrainTask.model, ckpt, logger)
        load_model_weight(TrainTask.teacher, ckpt, logger)
        logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))


[NanoDet][02-19 17:09:06]INFO:Starting task1
[NanoDet][02-19 17:09:06]INFO:Starting task1
[NanoDet][02-19 17:09:06]INFO:Starting task1
INFO:NanoDet:Starting task1
[NanoDet][02-19 17:09:06]INFO:Setting up data...
[NanoDet][02-19 17:09:06]INFO:Setting up data...
[NanoDet][02-19 17:09:06]INFO:Setting up data...
INFO:NanoDet:Setting up data...


cfg/task1.yml


Load 273 xml files and 625 boxes
creating index...
index created!


[NanoDet][02-19 17:09:06]INFO:Creating models
[NanoDet][02-19 17:09:06]INFO:Creating models
[NanoDet][02-19 17:09:06]INFO:Creating models
INFO:NanoDet:Creating models


Load 4530 xml files and 13606 boxes
creating index...
index created!
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


[NanoDet][02-19 17:09:07]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-19 17:09:07]INFO:Loaded model weight from models/task0/model_last.ckpt
[NanoDet][02-19 17:09:07]INFO:Loaded model weight from models/task0/model_last.ckpt
INFO:NanoDet:Loaded model weight from models/task0/model_last.ckpt


In [15]:
for batch in val_dataloader:
    batch = TrainTask._preprocess_batch_input(batch)
    break

In [12]:
gt_boxes = batch["gt_bboxes"]
print(gt_boxes)

[array([[140.72072, 185.58559, 171.42342, 211.17117]], dtype=float32), array([[158.72, 101.12, 231.68, 186.88],
       [ 97.92,  98.56, 132.48, 160.  ],
       [ 73.6 ,  90.24,  90.24, 120.96],
       [ 91.52,  85.76, 108.8 , 112.64]], dtype=float32), array([[ 28.05988  ,  39.47006  , 151.14072  , 218.67065  ],
       [198.33234  , 140.23055  , 213.       , 198.26347  ],
       [191.9551   , 144.69461  , 200.88324  , 188.05988  ],
       [177.92516  , 147.88324  , 189.40419  , 181.0449   ],
       [169.63474  , 146.60779  , 180.47604  , 187.42215  ],
       [183.66467  , 147.24551  , 195.78143  , 154.89821  ],
       [  1.2754492, 151.07185  ,  29.973053 , 161.91318  ],
       [  0.6377246, 153.62276  ,  16.58084  , 168.29042  ]],
      dtype=float32), array([[202.08981  ,  98.847305 , 309.86526  , 157.51796  ],
       [117.27245  ,  87.36826  , 215.48204  , 149.86526  ],
       [ 31.17964  ,  88.643715 , 108.344315 , 151.14072  ],
       [  2.4820359,  85.45509  ,  38.832336 , 137.748

In [16]:
img = batch["img"]
stud_feat = TrainTask.model.backbone(img)
stud_fpn_feat = TrainTask.model.fpn(stud_feat)
head_out = TrainTask.model.head(stud_fpn_feat)
dets = TrainTask.model.head.post_process(head_out, batch)

In [18]:
import numpy as np
id = 0
for img_id, img_dets in dets.items():
    boxes = []
    labels = []
    for label, bboxes in img_dets.items():
        for bbox in bboxes:
            score = bbox[-1]
            if score > 0.3:
                x0, y0, x1, y1 = [int(i) for i in bbox[:4]]
                boxes.append([x0, y0, x1, y1])
                labels.append(label)
    
    batch["gt_bboxes"][id] = np.append(batch["gt_bboxes"][id],np.array(boxes,dtype=np.float32))
    batch["gt_labels"][id] = np.append(batch["gt_labels"][id],np.array(labels))
    id += 1
print(len(batch["gt_bboxes"]))
        #batch["gt_bboxes"].extend(np.array(all_box, dtype=np.float32))

64


In [14]:
print(teach_int_head_out[1].shape)

torch.Size([1, 96, 16, 20])


In [30]:
import torch.nn as nn
stud_feat = TrainTask.model.backbone(img)
lista = []
lista1 = []
for tensor in stud_feat:
    tensor_prob = nn.functional.softmax(tensor, dim=1)
    lista.append(tensor)
    lista1.append(tensor_prob)
    print(tensor_prob.shape)
    print(tensor.shape)
mse_loss = nn.MSELoss()
for tensor1, tensor2 in zip(stud_feat,stud_feat):
    loss = mse_loss(tensor1, tensor2)
    print(loss)
    

torch.Size([1, 116, 32, 40])
torch.Size([1, 116, 32, 40])
torch.Size([1, 232, 16, 20])
torch.Size([1, 232, 16, 20])
torch.Size([1, 464, 8, 10])
torch.Size([1, 464, 8, 10])
tensor(0., grad_fn=<MseLossBackward0>)
tensor(0., grad_fn=<MseLossBackward0>)
tensor(0., grad_fn=<MseLossBackward0>)


In [27]:
import torch
import torch.nn.functional as F
outputs = []
for feat, cls_convs in zip(
    stud_fpn_feat,
    TrainTask.model.head.cls_convs,
):
    for conv in cls_convs:
        feat = conv(feat)
    outputs.append(feat)
    

mse_list = []
for tensor1, tensor2 in zip(outputs,outputs):
    mse = F.mse_loss(tensor1, tensor2)
    mse_list.append(mse.item())   
print(mse_list)
'''
    output = gfl_cls(feat)
    outputs.append(output.flatten(start_dim=2))
outputs = torch.cat(outputs, dim=2).permute(0, 2, 1)
'''

[0.0, 0.0, 0.0, 0.0]


'\n    output = gfl_cls(feat)\n    outputs.append(output.flatten(start_dim=2))\noutputs = torch.cat(outputs, dim=2).permute(0, 2, 1)\n'

In [10]:
teacher_state_dict = TrainTask.teacher.head.state_dict()
student_state_dict = TrainTask.model.head.state_dict()
print(TrainTask.teacher.head.cls_convs[0][1])

DepthwiseConvModule(
  (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
  (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1, inplace=True)
)


In [11]:
head = TrainTask.teacher.head

In [12]:
for conv in head.cls_convs:
    print(conv)

ModuleList(
  (0): DepthwiseConvModule(
    (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
    (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): DepthwiseConvModule(
    (depthwise): Conv2d(96, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=96, bias=False)
    (pointwise): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (dwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pwnorm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.1, inplace=True)
  )
)
ModuleList(
  (0): DepthwiseConvModule(
    (depthwise): Conv2d(96

In [3]:
##################
# COPY HEAD TEST #
##################
# Get the state dict of both models
teacher_state_dict = TrainTask.teacher.head.gfl_cls.state_dict()
student_state_dict = TrainTask.model.head.gfl_cls.state_dict()
# Update the specific layer weights in the student model with the teacher model weights
for name, param in teacher_state_dict.items():
    # Slice the weights tensor along the out_channels dimension from 0 to 15
    student_state_dict[name][:17] = param[:17]
# Load the updated state dict into the student model
TrainTask.model.head.gfl_cls.load_state_dict(student_state_dict)
trainer.save_checkpoint("models/task1/model_last.ckpt")